In [1]:
import numpy as np

## Edit distance

In [2]:
def ED(x: str, y: str) -> int:
    x, y = '$' + x, '$' + y
    mat = np.zeros((len(x), len(y)), dtype=int)
    for i in range(0, len(x)): mat[i][0] = i
    for j in range(0, len(y)): mat[0][j] = j
    for i in range(1, len(x)):
        for j in range(1, len(y)):
            if x[i] != y[j]:
                mat[i][j] = min(mat[i-1][j], mat[i-1][j-1], mat[i][j-1]) + 1
            else: 
                mat[i][j] = min(mat[i-1][j], mat[i-1][j-1], mat[i][j-1])
    return mat[len(x)-1][len(y)-1]

In [3]:
print(f"ED(breiburg, freiburg) = {ED('freiburg', 'breiburg')}")
print(f"ED(kid, kind) = {ED('kid', 'kind')}")
print(f"ED(cat, wildcat) = {ED('cat', 'wildcat')}")

ED(breiburg, freiburg) = 1
ED(kid, kind) = 1
ED(cat, wildcat) = 4


## Prefix edit distance

In [4]:
def PED(x: str, y: str, delta: int) -> int:
    if len(x) + delta + 1 <= len(y):
        x, y = '$' + x, '$' + y[:len(x) + delta]
        mat = np.zeros((len(x), len(y)), dtype=int)
        for i in range(0, len(x)): mat[i][0] = i
        for j in range(0, len(y)): mat[0][j] = j
        for i in range(1, len(x)):
            for j in range(1, len(y)):
                if x[i] != y[j]:
                    mat[i][j] = min(mat[i-1][j], mat[i-1][j-1], mat[i][j-1]) + 1
                else: 
                    mat[i][j] = min(mat[i-1][j], mat[i-1][j-1], mat[i][j-1])
        return mat[len(x)-1][len(y)-1]
    return -1

In [5]:
print(f"PED(Fibu, Freiburg) = {PED('Fibu', 'Freiburg', 2)}")
print(f"PED(aff, baff) = {PED('aff', 'baff', 1)}")

PED(Fibu, Freiburg) = 2
PED(aff, baff) = -1


## Q-Grams

In [6]:
def compute_qgram(x: str, q: int, padding: int = 3) -> list[str]:
    # Add padding to x. 
    x_padded = padding * '$' + x + padding * '$'
    qgram = [x_padded[i:i+q] for i in range(0, len(x_padded)) if len(x_padded[i:i+q]) == q] 
    return qgram

In [7]:
print(compute_qgram('freiburg', q=3, padding=0))
print(compute_qgram('freiburg', q=3, padding=2))

['fre', 'rei', 'eib', 'ibu', 'bur', 'urg']
['$$f', '$fr', 'fre', 'rei', 'eib', 'ibu', 'bur', 'urg', 'rg$', 'g$$']


### Similar words have many q-grams in common

In [8]:
print(compute_qgram('freeburger', q=2, padding=0))
print(compute_qgram('breeburger', q=2, padding=0))

['fr', 're', 'ee', 'eb', 'bu', 'ur', 'rg', 'ge', 'er']
['br', 're', 'ee', 'eb', 'bu', 'ur', 'rg', 'ge', 'er']


In [9]:
def intersect(list1: list, list2: list) -> list:
    set1 = set(list1)
    set2 = set(list2)
    return list(set1.intersection(set2))

In [10]:
qgram1 = compute_qgram('freeburger', q=2, padding=0)
qgram2 = compute_qgram('breeburger', q=2, padding=0)
inter = intersect(qgram1, qgram2)
print(inter)

['bu', 'ur', 'rg', 'ee', 're', 'ge', 'eb', 'er']


In [11]:
def merge_lists(list1: list, list2: list) -> list:
    i, j, n, m = 0, 0, len(list1), len(list2)
    merged = []
    while i < n and j < m:
        if list1[i][0] < list2[j][0]: merged.append(list1[i]); i+=1
        elif list1[i][0] > list2[j][0]: merged.append(list2[j]); j+=1
        else:
            new_score = list1[i][1] + list2[j][1]
            merged.append((list1[i][0], new_score))
            i += 1
            j += 1
    for x in list1[i:]: merged.append(x)
    for y in list2[j:]: merged.append(y)
    return merged
print(merge_lists([(1, 2), (3, 1), (5, 1)], [(2, 1), (3, 2), (9, 2)]))
print(merge_lists([(1, 2), (3, 1), (5, 1)], []))
print(merge_lists([], []))

[(1, 2), (2, 1), (3, 3), (5, 1), (9, 2)]
[(1, 2), (3, 1), (5, 1)]
[]


## Bring all together

In [12]:
import pandas as pd
import re

class FuzzySearch:

    def __init__(self, q: int, delta: int=1) -> None:
        # Map qgrams to (doc_id, tf-score)
        # tf-score = number of times word occurs in document.
        self.inverted_lists: dict[str, list[tuple[int, int]]] = {}

        # Save city names
        self.city_names: list[str] = []
        self.city_population: list[int] = []

        self.city_norm_names: list[str] = []

        self.q = q
        self.padding = q - 1
        self.delta = delta

    def tokenize(self, text: str) -> list[str]:
        WORD_PATTERN = '[a-zA-Z]+'
        return re.findall(WORD_PATTERN, str(text).lower())

    def norm(self, text: str) -> str:
        return text.lower()

    def build_from_file(self, file: str) -> None:
        cities = pd.read_csv(file)
        doc_id: int = 1 
        for _, row in cities.iterrows():
            for word in self.tokenize(row['name']):
                for qgram in compute_qgram(word, self.q, self.padding):
                    if qgram not in self.inverted_lists:
                        self.inverted_lists[qgram] = [(doc_id, 1)]
                    else:
                        if self.inverted_lists[qgram][-1][0] == doc_id:
                            new_tf = self.inverted_lists[qgram][-1][1] + 1
                            self.inverted_lists[qgram].append((doc_id, new_tf))
                        else:
                            self.inverted_lists[qgram].append((doc_id, 1))
            self.city_names.append(row['name'])
            self.city_norm_names.append(self.norm(row['name']))
            self.city_population.append(row['population']) 
            doc_id += 1

    def intersect(self, qgram1: list[str], qgram2: list[str]) -> list[str]:
        qset1 = set(qgram1)
        qset2 = set(qgram2)
        inter = list(qset1.intersection(qset2))
        return inter
    
    def process_query(self, query: str):
        query = self.norm(query) 
        query_gram = compute_qgram(query, self.q, self.padding)
        
        lists = []
        for qgram in query_gram:
            if qgram in self.inverted_lists: 
                intersect = [pair for pair in self.inverted_lists[qgram]]
                lists.append(intersect) 
        
        merged = []
        for x in lists:
            merged = merge_lists(merged, x)
        
        matches = []
        for doc_id, tf in merged:
            ped = PED(query, self.city_norm_names[doc_id-1], self.delta)
            if 0 < ped <= self.delta:
                matches.append((doc_id, ped))
        return matches

    def generate_output(self, query, threshold: int = 10):
        results = self.process_query(query)
        print(f'Query: {query}') 
        for i, (doc_id, ped) in enumerate(results):
            print(self.city_names[doc_id-1])
            if i > threshold:
                break

### Query: Frei

In [13]:
SearchEngine = FuzzySearch(q=2, delta=1)
SearchEngine.build_from_file('german_cities.csv')
SearchEngine.generate_output('Frei')

Query: Frei
Freie Hansestadt Bremen
Freie Hansestadt Bremen
Freiburg im Breisgau
Freiburg im Breisgau
Freising
Freiberg
Freital
Freilassing
Freiberg am Neckar
Pfreimd
Freinsheim
Freinsheim


### Query: Be

In [14]:
SearchEngine.generate_output('Be')

Query: Be
Berlin
Bremen
Bielefeld
Oberhausen
Bremerhaven
Bergisch Gladbach
Bergheim
Bergkamen
Oberursel (Taunus)
Bietigheim-Bissingen
Bietigheim-Bissingen
Bernau bei Berlin
